<a href="https://colab.research.google.com/github/MathewBiddle/sandbox/blob/main/IOOS_data_at_NCEI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using NCEI geoportal REST API to collect information about IOOS Regional Association archived data

Created: 2017-06-12

IOOS regional associations archive their non-federal observational data with NOAA's National Center for Environmental Information (NCEI). In this notebook we will use the [RESTful](https://github.com/Esri/geoportal-server/wiki/REST-API-Syntax) services of the [NCEI geoportal](https://www.ncei.noaa.gov/metadata/geoportal/#searchPanel) to collect metadata from the archive packages found in the NCEI archives. The metadata information are stored in [ISO 19115-2](https://wiki.earthdata.nasa.gov/display/NASAISO/ISO+19115-2) xml files which the NCEI geoportal uses for discovery of Archival Information Packages (AIPs). This example uses the ISO metadata records to display publication information as well as plot the time coverage of each AIP at NCEI which meets the search criteria.

First we update the namespaces dictionary from owslib to include the appropriate namespace reference for gmi and gml. 

For more information on ISO Namespaces see:  https://geo-ide.noaa.gov/wiki/index.php?title=ISO_Namespaces

In [1]:
try:
  import owslib
except:
  !pip install owslib

In [2]:
from owslib.iso import namespaces

# Append gmi namespace to namespaces dictionary.
namespaces.update({"gmi": "http://www.isotc211.org/2005/gmi"})
namespaces.update({"gml": "http://www.opengis.net/gml/3.2"})
del namespaces[None]

## Now we select a Regional Association and platform
This is where the user identifies the Regional Association and the platform type they are interested in. Change the RA acronym to the RA of interest. The user can also omit the Regional Association, by using `None`, to collect metadata information about all IOOS non-Federal observation data archived through the NCEI-IOOS pipeline.

The options for platform include: `"HF Radar"`, `"Glider"`, and `"FIXED PLATFORM"`. 

In [3]:
# Select RA, this will be the acronym for the RA or None if you want to search across all RAs
ra = None

# Identify the platform.
platform = '"FIXED PLATFORM"' # Options include: None, "HF Radar", "Glider", "FIXED PLATFORM"

## Next we generate a geoportal query and georss feed
To find more information about how to compile a geoportal query, have a look at [REST API Syntax](https://github.com/Esri/geoportal-server/wiki/REST-API-Syntax) and the [NCEI Search Tips](https://www.nodc.noaa.gov/search/granule/catalog/searchtips/searchtips.page) for the [NCEI geoportal](https://data.nodc.noaa.gov/geoportal/catalog/search/search.page). The example provided is specific to the NCEI-IOOS data pipeline project and only searches for non-federal timeseries data collected by each Regional Association. 

The query developed here can be updated to search for any Archival Information Packages at NCEI, therefore the user should develop the appropriate query using the [NCEI Geoportal](https://data.nodc.noaa.gov/geoportal/catalog/search/search.page) and update this portion of the code to identify the REST API of interest.

In [4]:
try:
    from urllib.parse import quote
except ImportError:
    from urllib import quote

# Generate geoportal query and georss feed.

# Base geoportal url.
baseurl = "https://www.ncei.noaa.gov/" "metadata/geoportal/opensearch" "?q="

# Identify the Regional Association
if ra is None:
    reg_assoc = ''
else:
    RAs = {
        "AOOS": "Alaska Ocean Observing System",
        "CARICOOS": "Caribbean Coastal Ocean Observing System",
        "CeNCOOS": "Central and Northern California Coastal Ocean Observing System",
        "GCOOS": "Gulf of Mexico Coastal Ocean Observing System",
        "GLOS": "Great Lakes Observing System",
        "MARACOOS": "Mid-Atlantic Regional Association Coastal Ocean Observing System",
        "NANOOS": "Northwest Association of Networked Ocean Observing Systems",
        "NERACOOS": "Northeastern Regional Association of Coastal Ocean Observing System",
        "PacIOOS": "Pacific Islands Ocean Observing System",
        "SCCOOS": "Southern California Coastal Ocean Observing System",
        "SECOORA": "Southeast Coastal Ocean Observing Regional Association",
        }
    reg_assoc = '(dataThemeinstitutions_s:"%s" dataThemeprojects_s:"%s (%s)")'%(RAs[ra], RAs[ra], ra)

# Identify the project.
project = '"Integrated Ocean Observing System Data Assembly Centers Data Stewardship Program"'

# Identify the amount of records and format of the response: 1 to 1010 records.
records = "&start=1&num=1010"

# Identify the format of the response: georss.
response_format = "&f=csv"

if platform is not None:
  if ra is not None:
    reg_assoc_plat = quote(reg_assoc + ' AND' + platform)
  else:
    reg_assoc_plat = quote(platform)
else:
  reg_assoc_plat = quote(reg_assoc)

# Combine the URL.
url = "{}{}{}{}".format(baseurl , reg_assoc_plat, '&filter=dataThemeprojects_s:', quote(project) + records + response_format)

print("Identified response format:\n{}".format(url))
print(
    "\nSearch page response:\n{}".format(url.replace(response_format, "&f=searchPage"))
)

Identified response format:
https://www.ncei.noaa.gov/metadata/geoportal/opensearch?q=%22FIXED%20PLATFORM%22&filter=dataThemeprojects_s:%22Integrated%20Ocean%20Observing%20System%20Data%20Assembly%20Centers%20Data%20Stewardship%20Program%22&start=1&num=1010&f=csv

Search page response:
https://www.ncei.noaa.gov/metadata/geoportal/opensearch?q=%22FIXED%20PLATFORM%22&filter=dataThemeprojects_s:%22Integrated%20Ocean%20Observing%20System%20Data%20Assembly%20Centers%20Data%20Stewardship%20Program%22&start=1&num=1010&f=searchPage


## Time to query the portal and parse out the csv response
Here we are opening the specified REST API and parsing it into a string. Then, since we identified it as a csv format above, we parse it using the Pandas package. We also split the Data_Date_Range column into two columns, `data_start_date` and `data_end_date` to have that useful information available.

In [5]:
import pandas as pd
import numpy as np

df = pd.read_csv(url)

df[['data_start_date','data_end_date']] = df['Data_Date_Range'].str.split(' to ',expand=True)
df['data_start_date'] = pd.to_datetime(df['data_start_date'])
df['data_end_date'] = pd.to_datetime(df['data_end_date']) + pd.Timedelta(np.timedelta64(1, "ms")) 

df.head()

,Id,Title,Description,West,South,East,North,Link_Xml,Link_1,Link_2,Link_3,Link_4,Data_Date_Range,Date_Published,data_start_date,data_end_date
0,gov.noaa.nodc:0172588,"Biological, chemical, and other data collected...",NaN,-124.19652,40.77750,-124.19652,40.77750,http://www.ncei.noaa.gov/metadata/geoportal/re...,NaN,NaN,NaN,NaN,2012-12-13T00:00:00Z to 2021-09-24T23:59:59.999Z,2018-04-18T00:00:00Z,2012-12-13 00:00:00+00:00,2021-09-25 00:00:00+00:00
1,gov.noaa.nodc:0172612,"Biological, chemical, and other data collected...",NaN,-121.88935,36.60513,-121.88935,36.60513,http://www.ncei.noaa.gov/metadata/geoportal/re...,NaN,NaN,NaN,NaN,2015-05-05T00:00:00Z to 2020-01-03T23:59:59.999Z,2018-04-18T00:00:00Z,2015-05-05 00:00:00+00:00,2020-01-04 00:00:00+00:00
2,gov.noaa.nodc:0172613,"Biological, chemical, and other data collected...",NaN,-124.15754,40.81503,-124.15754,40.81503,http://www.ncei.noaa.gov/metadata/geoportal/re...,NaN,NaN,NaN,NaN,2016-04-05T00:00:00Z to 2019-10-28T23:59:59.999Z,2018-04-18T00:00:00Z,2016-04-05 00:00:00+00:00,2019-10-29 00:00:00+00:00
3,gov.noaa.nodc:0163602,Oceanographic and surface meteorological data ...,NaN,-73.90430,40.93626,-73.90430,40.93626,http://www.ncei.noaa.gov/metadata/geoportal/re...,NaN,NaN,NaN,NaN,2014-06-12T00:00:00Z to 2017-10-20T23:59:59.999Z,2017-06-21T00:00:00Z,2014-06-12 00:00:00+00:00,2017-10-21 00:00:00+00:00
4,gov.noaa.nodc:0163603,Oceanographic data collected from station Mari...,NaN,-73.93880,41.72060,-73.93880,41.72060,http://www.ncei.noaa.gov/metadata/geoportal/re...,NaN,NaN,NaN,NaN,2014-01-10T00:00:00Z to 2019-03-31T23:59:59.999Z,2017-06-21T00:00:00Z,2014-01-10 00:00:00+00:00,2019-04-01 00:00:00+00:00


Now, lets pull out all the ISO metadata record links and print them out so the user can browse to the metadata record and look for what items they might be interested in.

In [6]:
# parse the csv response

print("Found %i record(s)" % len(df))
# for index, row in df.iterrows():
#     print('ISO19115-2 record:',row['Link_Xml'])  # URL to ISO19115-2 record.
#     print('NCEI dataset metadata page: https://www.ncei.noaa.gov/access/metadata/landing-page/bin/iso?id=' + row['Id'] )
#     print('\n')

Found 214 record(s)


## Let's collect what we have found
Now that we have all the ISO metadata records we are interested in, it's time to do something fun with them. In this example we want to generate a timeseries plot of the data coverage for the "Southern California Coastal Ocean Observing System" stations we have archived at NCEI.

First we need to collect some information. We loop through each iso record to collect metadata information about each package. The example here shows how to collect the following items:
   1. NCEI Archival Information Package (AIP) Accession ID (7-digit Accession Number) 
   2. The first date the archive package was published.
   3. The platform code identified from the provider.
   4. The version number and date it was published.
   5. The current AIP size, in MB.

There are plenty of other metadata elements to collect from the ISO records, so we recommend browsing to one of the records and having a look at the items of interest to your community.

```
<gmd:onLine>
<gmd:CI_OnlineResource>
<gmd:linkage>
<gmd:URL>https://www.ncei.noaa.gov/thredds-ocean/catalog/ioos/glos/ESF1/catalog.html</gmd:URL>
</gmd:linkage>
<gmd:protocol>
<gco:CharacterString>THREDDS</gco:CharacterString>
```

In [7]:
# Process each iso record.
%matplotlib inline

from datetime import datetime
import xml.etree.ElementTree as ET
from owslib import util
from urllib.request import urlopen

df_out = pd.DataFrame()

df[['provider_platform_name','NCEI_accession_number','package_size_mb','submitter']] = ''

# For each accession in response.
for url in df['Link_Xml']:

    iso = urlopen(url)
    iso_tree = ET.parse(iso)
    root = iso_tree.getroot()

    for distribution in root.iterfind('.//gmd:onLine', namespaces):

      for service in distribution.iterfind('.//gmd:protocol/gco:CharacterString', namespaces):

        if service.text == "THREDDS":

          df1 = pd.DataFrame({'thredds': [distribution.find('.//gmd:URL', namespaces).text]})
          df_out = pd.concat([df_out, df1])
    

In [8]:
try:
  import siphon
except:
  !pip install siphon

## search for CF standard names of interest

* `fractional_saturation_of_oxygen_in_sea_water`
* `mass_concentration_of_oxygen_in_sea_water`
* `mole_concentration_of_dissolved_molecular_oxygen_in_sea_water`
* `mole_concentration_of_dissolved_molecular_oxygen_in_sea_water_at_saturation`
* `mole_concentration_of_dissolved_molecular_oxygen_in_sea_water_at_shallowest_local_minimum_in_vertical_profile`
* `volume_fraction_of_oxygen_in_sea_water`

In [32]:
from siphon.catalog import TDSCatalog
import xarray as xr

df_final = pd.DataFrame()

std_names = ['fractional_saturation_of_oxygen_in_sea_water',
            'mass_concentration_of_oxygen_in_sea_water',
            'mole_concentration_of_dissolved_molecular_oxygen_in_sea_water',
            'mole_concentration_of_dissolved_molecular_oxygen_in_sea_water_at_saturation',
            'mole_concentration_of_dissolved_molecular_oxygen_in_sea_water_at_shallowest_local_minimum_in_vertical_profile',
            'volume_fraction_of_oxygen_in_sea_water']

# for each accession
for index, row in df_out.iterrows():

  try:
    TDSCatalog(row['thredds'])
  except:
    print('invalid url:',row['thredds'])
    continue

  # look through THREDDS catalog
  print('\nTHREDDS Catalog:', row['thredds'])

  catalog = TDSCatalog(row['thredds'])

  # for each dataset in thredds catalog
  for name, ds in catalog.datasets.items():

      # if data is accessible
      if ds.access_urls:
          #print(name)
          link = ds.access_urls['OPENDAP']
          print('dataset:',name)
          if name in ['STCPBY1_d1_201706.nc','STCPBY1_d1_201705.nc']:
             continue
          # open the data
          dset = xr.open_dataset(link+'#fillmismatch') # avoids fill mismatch error

          # check if standard name is in standard name list.
          for name in std_names:
            var = dset.filter_by_attrs(standard_name=name)
            if len(var.data_vars) > 0:
              print('Found match for:',name)
              print('link:',link)
              print(var.data_vars)
              
              df_in = pd.DataFrame({'thredds_link': [link],
                                    'variable':[var.data_vars],
                                    'standard_name':[name],
                                    })
              
              df_final = pd.concat([df_final, df_in])
            #print(var)
# info = """
# Catalog information
# -------------------

# Base THREDDS URL: {}
# Catalog name: {}
# Catalog URL: {}
# Metadata: {}
# """.format(
#     catalog.base_tds_url, catalog.catalog_name, catalog.catalog_url, catalog.metadata
# )

# print(info)



invalid url: https://www.ncei.noaa.gov/thredds-ocean/catalog/ioos/cencoos/edu_humboldt_humboldt/catalog.html
invalid url: https://www.ncei.noaa.gov/thredds-ocean/catalog/ioos/cencoos/mlml_monterey/catalog.html
invalid url: https://www.ncei.noaa.gov/thredds-ocean/catalog/ioos/cencoos/wiyot_tribe_indianisland/catalog.html


/usr/local/lib/python3.7/dist-packages/siphon/catalog.py:293: UserWarning: URL https://www.ncei.noaa.gov/thredds-ocean/catalog/ioos/maracoos/hrecos-waf/HRBZAK/catalog.html returned HTML. Changing to: https://www.ncei.noaa.gov/thredds-ocean/catalog/ioos/maracoos/hrecos-waf/HRBZAK/catalog.xml
  new_url))



THREDDS Catalog: https://www.ncei.noaa.gov/thredds-ocean/catalog/ioos/maracoos/hrecos-waf/HRBZAK/catalog.html
dataset: HRBZAK_d1_201710.nc
Found match for: fractional_saturation_of_oxygen_in_sea_water
link: https://www.ncei.noaa.gov/thredds-ocean/dodsC/ioos/maracoos/hrecos-waf/HRBZAK/HRBZAK_d1_201710.nc
Data variables:
    fractional_saturation_of_oxygen_in_sea_water  (time) float32 ...
Found match for: mass_concentration_of_oxygen_in_sea_water
link: https://www.ncei.noaa.gov/thredds-ocean/dodsC/ioos/maracoos/hrecos-waf/HRBZAK/HRBZAK_d1_201710.nc
Data variables:
    mass_concentration_of_oxygen_in_sea_water  (time) float32 ...
dataset: HRBZAK_d1_201709.nc
Found match for: fractional_saturation_of_oxygen_in_sea_water
link: https://www.ncei.noaa.gov/thredds-ocean/dodsC/ioos/maracoos/hrecos-waf/HRBZAK/HRBZAK_d1_201709.nc
Data variables:
    fractional_saturation_of_oxygen_in_sea_water  (time) float32 ...
Found match for: mass_concentration_of_oxygen_in_sea_water
link: https://www.ncei.no

/usr/local/lib/python3.7/dist-packages/siphon/catalog.py:293: UserWarning: URL https://www.ncei.noaa.gov/thredds-ocean/catalog/ioos/maracoos/hrecos-waf/HRMARPHS/catalog.html returned HTML. Changing to: https://www.ncei.noaa.gov/thredds-ocean/catalog/ioos/maracoos/hrecos-waf/HRMARPHS/catalog.xml
  new_url))



THREDDS Catalog: https://www.ncei.noaa.gov/thredds-ocean/catalog/ioos/maracoos/hrecos-waf/HRMARPHS/catalog.html
dataset: HRMARPHS_d1_201903.nc
Found match for: fractional_saturation_of_oxygen_in_sea_water
link: https://www.ncei.noaa.gov/thredds-ocean/dodsC/ioos/maracoos/hrecos-waf/HRMARPHS/HRMARPHS_d1_201903.nc
Data variables:
    fractional_saturation_of_oxygen_in_sea_water  (time) float32 ...
Found match for: mass_concentration_of_oxygen_in_sea_water
link: https://www.ncei.noaa.gov/thredds-ocean/dodsC/ioos/maracoos/hrecos-waf/HRMARPHS/HRMARPHS_d1_201903.nc
Data variables:
    mass_concentration_of_oxygen_in_sea_water  (time) float32 ...
dataset: HRMARPHS_d1_201902.nc
Found match for: fractional_saturation_of_oxygen_in_sea_water
link: https://www.ncei.noaa.gov/thredds-ocean/dodsC/ioos/maracoos/hrecos-waf/HRMARPHS/HRMARPHS_d1_201902.nc
Data variables:
    fractional_saturation_of_oxygen_in_sea_water  (time) float32 ...
Found match for: mass_concentration_of_oxygen_in_sea_water
link: h

...

/usr/local/lib/python3.7/dist-packages/siphon/catalog.py:293: UserWarning: URL https://www.ncei.noaa.gov/thredds-ocean/catalog/ioos/maracoos/hrecos-waf/HRWSTPTH/catalog.html returned HTML. Changing to: https://www.ncei.noaa.gov/thredds-ocean/catalog/ioos/maracoos/hrecos-waf/HRWSTPTH/catalog.xml
  new_url))



THREDDS Catalog: https://www.ncei.noaa.gov/thredds-ocean/catalog/ioos/maracoos/hrecos-waf/HRWSTPTH/catalog.html
dataset: HRWSTPTH_d2_201903.nc
Found match for: fractional_saturation_of_oxygen_in_sea_water
link: https://www.ncei.noaa.gov/thredds-ocean/dodsC/ioos/maracoos/hrecos-waf/HRWSTPTH/HRWSTPTH_d2_201903.nc
Data variables:
    fractional_saturation_of_oxygen_in_sea_water  (time) float32 ...
Found match for: mass_concentration_of_oxygen_in_sea_water
link: https://www.ncei.noaa.gov/thredds-ocean/dodsC/ioos/maracoos/hrecos-waf/HRWSTPTH/HRWSTPTH_d2_201903.nc
Data variables:
    mass_concentration_of_oxygen_in_sea_water  (time) float32 ...
dataset: HRWSTPTH_d2_201812.nc
Found match for: fractional_saturation_of_oxygen_in_sea_water
link: https://www.ncei.noaa.gov/thredds-ocean/dodsC/ioos/maracoos/hrecos-waf/HRWSTPTH/HRWSTPTH_d2_201812.nc
Data variables:
    fractional_saturation_of_oxygen_in_sea_water  (time) float32 ...
Found match for: mass_concentration_of_oxygen_in_sea_water
link: h

/usr/local/lib/python3.7/dist-packages/siphon/catalog.py:293: UserWarning: URL https://www.ncei.noaa.gov/thredds-ocean/catalog/ioos/maracoos/hrecos-waf/STCPBY1/catalog.html returned HTML. Changing to: https://www.ncei.noaa.gov/thredds-ocean/catalog/ioos/maracoos/hrecos-waf/STCPBY1/catalog.xml
  new_url))



THREDDS Catalog: https://www.ncei.noaa.gov/thredds-ocean/catalog/ioos/maracoos/hrecos-waf/STCPBY1/catalog.html
dataset: STCPBY1_d1_201710.nc
Found match for: fractional_saturation_of_oxygen_in_sea_water
link: https://www.ncei.noaa.gov/thredds-ocean/dodsC/ioos/maracoos/hrecos-waf/STCPBY1/STCPBY1_d1_201710.nc
Data variables:
    fractional_saturation_of_oxygen_in_sea_water  (time) float32 ...
Found match for: mass_concentration_of_oxygen_in_sea_water
link: https://www.ncei.noaa.gov/thredds-ocean/dodsC/ioos/maracoos/hrecos-waf/STCPBY1/STCPBY1_d1_201710.nc
Data variables:
    mass_concentration_of_oxygen_in_sea_water  (time) float32 ...
dataset: STCPBY1_d1_201709.nc
Found match for: fractional_saturation_of_oxygen_in_sea_water
link: https://www.ncei.noaa.gov/thredds-ocean/dodsC/ioos/maracoos/hrecos-waf/STCPBY1/STCPBY1_d1_201709.nc
Data variables:
    fractional_saturation_of_oxygen_in_sea_water  (time) float32 ...
Found match for: mass_concentration_of_oxygen_in_sea_water
link: https://ww

OverflowError: ignored

In [14]:
#df_out.iloc[30]['thredds']
catalog = TDSCatalog('https://www.ncei.noaa.gov/thredds-ocean/dodsC/ioos/maracoos/hrecos-waf/HRSCHD/HRSCHDH_d1_200806_verified.nc')

for name, ds in catalog.datasets.items():
    if ds.access_urls:
        #print(name)
        link = ds.access_urls['OPENDAP']
        print(link)
        dset = xr.open_dataset(link+'#fillmismatch') # avoids fill mismatch error
        for name in std_names:
          #print('searching for:', name)
          var = dset.filter_by_attrs(standard_name=name)
          if len(var.data_vars) > 0:
            print('Found match for:',name)
            print(var.data_vars)

HTTPError: ignored

In [15]:
link = 'https://www.ncei.noaa.gov/thredds-ocean/dodsC/ioos/maracoos/hrecos-waf/HRSCHD/HRSCHDH_d1_200806_verified.nc'
dset = xr.open_dataset(link+'#fillmismatch') # avoids fill mismatch error
for name in std_names:
  #print('searching for:', name)
  var = dset.filter_by_attrs(standard_name=name)
  if len(var.data_vars) > 0:
    print('Found match for:',name)
    print(var.data_vars)

Found match for: fractional_saturation_of_oxygen_in_sea_water
Data variables:
    fractional_saturation_of_oxygen_in_sea_water  (time) float32 ...
Found match for: mass_concentration_of_oxygen_in_sea_water
Data variables:
    mass_concentration_of_oxygen_in_sea_water  (time) float32 ...


In [29]:
dset.coords['lat'].

AttributeError: ignored

## Create a timeseries plot of data coverage
Now that we have a DataFrame with all the information we're interested in, lets make a time coverage plot for all the AIP's at NCEI.

In [ ]:
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
    
ypos = range(len(df))
fig, ax = plt.subplots(figsize=(15, 12))

# Plot the data
ax.barh(ypos, mdates.date2num(df['data_end_date']) - mdates.date2num(df['data_start_date']), 
        left = mdates.date2num(df['data_start_date']), 
        height = 0.5, 
        align = 'center')

xlim = ( mdates.date2num(df['data_start_date'].min() - pd.Timedelta(np.timedelta64(1, "M"))),
         mdates.date2num(df['data_end_date'].max() + pd.Timedelta(np.timedelta64(1, "M"))) )

ax.set_xlim(xlim)
ax.set(yticks = np.arange(0, len(df)))
ax.tick_params(which="both", direction="out")
ax.set_ylabel("NCEI Accession Number")
ax.set_yticklabels(df['NCEI_accession_number'])
ax.set_title('NCEI archive package time coverage')

ax.xaxis_date()
ax.set_xlabel('Date')

plt.grid(axis='x', linestyle='--')

This procedure has been developed as an example of how to use NCEI's geoportal REST API's to collect information about packages that have been archived at NCEI. The intention is to provide some guidance and ways to collect this information without having to request it directly from NCEI. There are a significant amount of metadata elements which NCEI makes available through their ISO metadata records. Therefore, anyone interested in collecting other information from the records at NCEI should have a look at the ISO metadata records and determine which items are of interest to their community. Then, update the example code provided to collect that information.

**Author:** Mathew Biddle